In [2]:
import pandas as pd
import os
from openpyxl import load_workbook
from io import BytesIO
from PIL import Image as PILImage
from openpyxl.drawing.image import Image as OpenPyImage

# ------------ CONFIG ------------
RUTA_PROD = "data/NHMX511301944912-produdctos.xlsx"   # archivo original (NO se modifica)
RUTA_SALES = "data/sales.xlsx"                      # solo lectura
HOJA = "PCODE"
CARPETA_IMAGENES = "catalogo-img"
SALIDA_HTML = "catalogo_generado.html"
SALIDA_INVENTARIO = "inventario_actual.xlsx"        # NUEVO ARCHIVO DONDE SE GUARDA TODO
# ---------------------------------

os.makedirs(CARPETA_IMAGENES, exist_ok=True)

# 1) Leer productos (stock inicial)
df = pd.read_excel(RUTA_PROD, sheet_name=HOJA)

required = ["ProductCode", "Product Name", "Qty"]
for c in required:
    if c not in df.columns:
        raise SystemExit(f"Falta columna requerida: {c}")

# 2) Leer ventas sin modificar sales.xlsx
if os.path.exists(RUTA_SALES):
    ventas = pd.read_excel(RUTA_SALES)
    if "ProductCode" not in ventas.columns or "Quantity" not in ventas.columns:
        raise SystemExit("sales.xlsx debe tener columnas: ProductCode y Quantity")

    ventas_total = ventas.groupby("ProductCode")["Quantity"].sum()
    df["Ventas"] = df["ProductCode"].map(ventas_total).fillna(0).astype(int)
else:
    df["Ventas"] = 0

# 3) Calcular Inventario actual
df["Inventario"] = df["Qty"].astype(int) - df["Ventas"].astype(int)

print("\n⭐ Inventario procesado:\n", df[["ProductCode","Qty","Ventas","Inventario"]].head())

# 4) Guardar inventario en EXCEL independiente (NO se toca productos original)
df_inv = df[["ProductCode","Inventario"]]
df_inv.to_excel(SALIDA_INVENTARIO, index=False)
print(f"📁 Archivo inventario creado -> {SALIDA_INVENTARIO}")

# 5) Extraer imágenes del archivo original (sin alterarlo)
wb = load_workbook(RUTA_PROD, data_only=True)
ws = wb[HOJA]
imgs = getattr(ws, "_images", [])
print(f"\n📷 Imágenes detectadas: {len(imgs)}")

for img in imgs:
    try:
        row_excel = img.anchor._from.row + 1
    except:
        try:
            row_excel = img.anchor.row
        except:
            continue

    idx = row_excel - 2
    if idx < 0 or idx >= len(df):
        continue

    code = str(df.iloc[idx]["ProductCode"])
    name = f"{code}.png"
    path_out = os.path.join(CARPETA_IMAGENES, name)

    try:
        data = img._data() if hasattr(img,"_data") else img.ref
        if not data:
            raise Exception()

        pil = PILImage.open(BytesIO(data))
        pil.save(path_out)

        print(f"✔ Imagen guardada: {name}")

    except:
        if hasattr(img,"path") and img.path:
            from shutil import copyfile
            copyfile(img.path, path_out)
            print(f"✔ Copiada por path: {name}")
        else:
            print(f"⚠ No se pudo extraer imagen de {code}")

# 6) Generar HTML (solo Inventario)
html = "<div class='grid'>\n"

for _, row in df.iterrows():
    code = row["ProductCode"]
    name = row["Product Name"]
    desc = row["Product Name"]
    inventario = int(row["Inventario"])

    img = f"{code}.png"
    if not os.path.exists(f"{CARPETA_IMAGENES}/{img}"):
        img = f"{code}.jpg"

    html += f"""
    <div class="producto">
        <img src="{CARPETA_IMAGENES}/{img}" alt="{code}">
        <h3>{code} — {name}</h3>
        <p>{desc}</p>
        <span class="precio">📍 Inventario actual: <b>{inventario}</b></span>
    </div>
    """

html += "\n</div>"

with open(SALIDA_HTML, "w", encoding="utf-8") as f:
    f.write(html)

print(f"\n🎉 HTML generado -> {SALIDA_HTML}")
print("🟢 productos.xlsx NO fue modificado")
print("🔵 sales.xlsx NO fue tocado")
print("🟣 Inventario generado desde cero y limpio")



⭐ Inventario procesado:
   ProductCode  Qty  Ventas  Inventario
0       A0269    4       0           4
1       A0270    4       0           4
2       A0271    2       0           2
3       A0272    2       0           2
4       A0273    2       0           2
📁 Archivo inventario creado -> inventario_actual.xlsx

📷 Imágenes detectadas: 96
✔ Imagen guardada: A0270.png
✔ Imagen guardada: A0271.png
✔ Imagen guardada: A0272.png
✔ Imagen guardada: A0273.png
✔ Imagen guardada: A0274.png
✔ Imagen guardada: A0275.png
✔ Imagen guardada: A0276.png
✔ Imagen guardada: A0277.png
✔ Imagen guardada: A0278.png
✔ Imagen guardada: A0279.png
✔ Imagen guardada: A0280.png
✔ Imagen guardada: A0281.png
✔ Imagen guardada: A0282.png
✔ Imagen guardada: A0283.png
✔ Imagen guardada: A0284.png
✔ Imagen guardada: A0285.png
✔ Imagen guardada: A0286.png
✔ Imagen guardada: A0287.png
✔ Imagen guardada: A0288.png
✔ Imagen guardada: A0289.png
✔ Imagen guardada: A0290.png
✔ Imagen guardada: A0291.png
✔ Imagen guardada: A0